In [ ]:
import sounddevice as sd
import numpy as np
import serial
import time

# ---------------- CONFIG ----------------
DEVICE = None             # Default audio device (set index if needed)
SAMPLE_RATE = 44100       # Hz
BLOCK_SIZE = 1024         # Audio buffer size
SERIAL_PORT = '/dev/cu.usbmodem21301'      # Change for your system (e.g. '/dev/ttyACM0' on Linux/Mac)
BAUD_RATE = 115200

ATTACK_MS = 50            # Attack time in ms
RELEASE_MS = 200          # Release time in ms
UPDATE_INTERVAL = 0.05    # Send every 50ms
# ----------------------------------------

attack_coeff = np.exp(-1.0 / (ATTACK_MS * SAMPLE_RATE / 1000.0))
release_coeff = np.exp(-1.0 / (RELEASE_MS * SAMPLE_RATE / 1000.0))

level = 0.0   # Smoothed signal level
last_send = time.time()

# Print available audio devices
print("Available audio devices:\n")
print(sd.query_devices())
print("\n--- End of device list ---\n")

# Open serial
ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)

def audio_callback(indata, frames, time_info, status):
    global level, last_send
    
    # Convert to mono
    mono = np.mean(indata, axis=1)
    
    # RMS (root mean square) as level
    rms = np.sqrt(np.mean(mono**2))
    
    # Envelope follower with attack/release
    if rms > level:
        coeff = attack_coeff
    else:
        coeff = release_coeff
    
    level = (1 - coeff) * rms + coeff * level
    
    # Check if it's time to send
    now = time.time()
    if now - last_send >= UPDATE_INTERVAL:
        scaled = int(np.clip(level * 127 * 10, 0, 127))  # scaling factor
        print(scaled)
        ser.write(bytes([scaled]))
        last_send = now

# Open audio stream
with sd.InputStream(device=DEVICE,
                    channels=1,
                    samplerate=SAMPLE_RATE,
                    blocksize=BLOCK_SIZE,
                    callback=audio_callback):
    print("Streaming audio... Press Ctrl+C to stop.")
    try:
        while True:
            time.sleep(0.1)
    except KeyboardInterrupt:
        print("Stopped.")